### Álvaro Plata, Brenda Barahona, Sofía Álvarez (201729031)

In [1]:
# ESAI

# Importamos todas las librerias necesarias para el laboratorio.
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()  # for plot styling
%matplotlib inline
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

<h1 align='center'>Laboratorio 1: Inteligencia de Negocios</h1>

SaludAlpes es una entidad de salud colombiana que se especializa en atender pacientes diagnosticados con diabetes.
Entre las actividades principales de SaludAlpes se destaca el análisis de resultados de laboratorio para la detección de diabetes.
Este servicio de diagnóstico implica el análisis manual por parte de médicos especialistas, a partir de los resultados de exámenes de laboratorio, lo cual ha generado en los últimos años, una serie de problemas asociados con los tiempos de atención a estos pacientes, causada por demoras en la confirmación del diagnóstico.
SaludAlpes los ha contratado para agilizar y mejorar el proceso de análisis de resultados clínicos de tal manera que se reduzcan los tiempos de confirmación de diagnóstico y se agilice el inicio del tratamiento a paciente con diabetes confirmada.
Su labor como consultor de BI es agilizar el proceso de confirmación de diagnóstico de diabetes en un paciente, utilizando datos históricos y realizando sobre ellos procesos de análisis y procesamiento, al igual que comunicando los resultados a la entidad para afianzar sus conocimientos en áreas de aprendizaje automático y al mismo tiempo generar una ventaja competitiva.
Con el fin de lograr el objetivo para el cual fue contratado, SaludAlpes espera tener tres modelos distintos de clasificación, construidos con 3 técnicas diferentes, entre las cuales deben estar árboles de decisión y KNN (k-nearest-neighbours). De igual manera, espera que siga la metodología que ellos utilizan en la actualidad para el desarrollo de ese tipo de proyectos,
para lo cual le sugiere realizar los siguientes pasos:

### 1. Análisis exploratorio y perfilamiento de los datos.

Para poder realizar un buen algoritmo de Machine Learning, lo primero que debemos hacer es analizar y perfilar los datos, con el fin de prepararlos para 

Lo primero que hacemos es leer los datos:

In [2]:
datos = pd.read_csv('202210_Laboratorio1_data_Datos_Clasificacion_2022.csv', delimiter=';', low_memory=False)

Cabe anotar que en este caso, debemos poner el atributo de baja memoria en False. Un warning decía que hay tipos de datos mixtos y, para evitarlo, debíamos hacer eso con el parámetro de la función <code>pd.read_csv()</code>.

Una vez leídos los datos, podemos ver la cantidad de columnas (features) y datos (filas) que tiene nuestro conjunto de datos:

In [3]:
datos.shape

(100000, 27)

En principio, nuestro conjunto de datos cuenta con 100.000 registros y 27 columnas. Veamos una muestra aleatoria de 5 datos del dataset:

In [4]:
display(datos.sample(5))

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,DiffWalk,Sex,Age,Education,Income,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
3046,0,0,0,1,20,1,0,0,1,1,...,0,1,13,6,7,NaN,NaN,NaN,NaN,NaN
9348,0,0,0,1,35,1,0,0,1,1,...,0,0,6,6,8,NaN,NaN,NaN,NaN,NaN
60082,0,0,0,1,23,0,0,0,1,1,...,0,0,8,4,7,NaN,NaN,NaN,NaN,NaN
34640,0,0,0,1,24,1,0,0,1,0,...,0,1,2,5,5,NaN,NaN,NaN,NaN,NaN
42905,2,1,0,1,27,1,0,0,1,0,...,1,1,11,5,6,NaN,NaN,NaN,NaN,NaN


En principio, podemos intuir que la variable objetivo será <code>Diabetes_012</code>, porque de acuerdo con el diccionario del negocio, es la que nos permitirá saber si un paciente tiene diabetes, prediabetes, o ninguna condición, como es del interés de SaludAlpes. Antes de seguir con el preprocesamiento, hacemos la partición train_test, con train=80% de los datos y test=20%. No tomamos conjunto de validación pues haremos validación cruzada usando GridSearch para todos los modelos. Eliminamos la variable <code>Diabetes_012</code> de X_test y X_train y las convertimos en Y_test y Y_train, respectivamente.

In [5]:
Y = datos['Diabetes_012']

X = datos.drop(['Diabetes_012'], axis=1)

X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2, random_state=28)
Y

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: Diabetes_012, Length: 100000, dtype: object

Volvemos a juntar las X, Y de entrenamiento para poder eliminar más adelante los registros completos de algunos datos, como se verá más adelante:

In [6]:
datos_train = X_train.assign(Diabetes_012 = Y)

Podemos ver el tamaño de nuestro nuevo conjunto de entrenamiento:

In [7]:
print('El tamaño del conjunto de entrenamiento es: {}'.format(len(datos_train)))

El tamaño del conjunto de entrenamiento es: 80000


**Nota:** Es importante anotar que lo primero que intentamos hacer fue un _pandas profiling_. No obstante, debido a que hay diversos tipos de datos en las columnas, este arroja varios errores. Por ello, debemos realizar un análisis exploratorio y corregir estas cuestiones antes de utilizar la herramienta.

Podemos ver que al final de esta muestra hay 5 columnas que aparecen sin nombre: Unnamed:22 hasta Unnamed:26, cuyos valores están todos en NaN. Por ello, decidimos revisar las otras columnas del dataframe y compararlas con el diccionario de datos dado para este laboratorio.

In [8]:
print(list(datos_train.columns))

['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Diabetes_012']


Contrastando los nombres de las columnas con el diccionario, vemos que las columnas 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25' y 'Unnamed: 26' no están caracterizadas por el negocio y, por tanto, no nos dan información relevante para el modelo. Más aún, podemos ver los valores únicos de cada una de estas columnas.

In [9]:
# Iteramos de 22 a 26 para conocer los valores únicos de cada una de las columnas Unnamed.
for i in range(22,27):
    print(datos_train['Unnamed: {num}'.format(num=i)].unique(), end=' ')

[nan] [nan] [nan] [nan] [nan] 

Vemos entonces que estas columnas están llenas de <code>NaN</code>; i.e. Not a Number. Por tanto, no aportan nada y procedemos a eliminarlas de nuestro conjunto de datos. Para ello, hacemos todo en funciones, con el fin de poder aplicar este pre-procesamiento al test también.

In [10]:
def eliminar_columnas_unnamed(df, col_inicial, col_final):
    # Eliminamos las últimas columnas
    return df.drop(['Unnamed: {num}'.format(num=i) for i in range(col_inicial, col_final+1)], axis=1)

In [11]:
datos_train = eliminar_columnas_unnamed(datos_train, 22, 26)

Revisamos que las columnas hayan sido correctamente eliminadas:

In [12]:
print(list(datos_train.columns))

['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income', 'Diabetes_012']


Ahora, ya tenemos todas las columnas que se encuentran en el diccionario. Lo siguiente que queremos ver es explorar los datos que hay en nuestro dataset. Nos preguntamos: ¿Por qué, al comienzo, tuvimos un _warning_ con los tipos de datos? ¿Qué datos están almacenados en el conjunto?

Revisando el diccionario de datos, nos dimos cuenta de que todos los atributos debían ser numéricos (así fueran categorías numéricas - como es el caso de columnas como GenHlth, Age y Education, entre otras - o binarias). Se nos ocurrió, entonces, revisar si las columnas estaban presentando otro tipo de datos no numéricos. Para ello, primero buscamos en el conjunto de datos todos los valores distintos que hay en nuestro dataframe y los convertimos en una lista. Luego, iteramos sobre ellos e intentamos convertirlos (hacer el cast) a números flotantes, usando un bloque try-except. Si se lanza una excepción, significa que el dato no es numérico y lo agregamos a la lista, para saber qué otro tipo de datos tiene nuestro dataframe.

In [13]:
# Vemos todos los valores distintos (únicos) que tiene el dataframe.
valores_distintos = list(datos_train.apply(pd.value_counts).index)
valores_no_numericos = []
# Interamos sobre los valores distintos
for val in valores_distintos:
    try:
        # Intentamos hacer el cast a float
        numero = float(val)
    except:
        # Si hay datos no numericos, los agregamos a la lista
        valores_no_numericos.append(val)
print('Los valores no numéricos son: {}'.format(valores_no_numericos))

Los valores no numéricos son: ['-', '?', 'Xx']


Del procedimiento anterior, podemos ver que hay 3 tipos de datos no numéricos en todo el dataframe.

Debemos tomar una decisión sobre si completamos estos datos con medidas de tendencia central (en este caso, como la moda, porque son atributos categóricos), o si los eliminamos. Para tomar estas decisiones, debemos saber en cuántos datos del conjunto total hay estas inconsistencias. 

Es importante anotar que vamos fila por fila en el conjunto de datos porque, si revisamos en columnas (por ejemplo, usando las funcionalidades de <code>pandas</code>: <code>datos[columna] == '-'</code>) puede ocurrir que, para el mismo dato en otra columna, haya otro '-', o una 'Xx' y un '?'. Esto es difícil saberlo en pandas y, por ello, recorremos todo el conjunto de datos.

In [14]:
num_inconsistentes = len(datos_train[((datos_train.values == '-')|(datos_train.values == '?')|(datos_train.values == 'Xx')).any(axis=1)])
print('El número de filas con datos inconsistentes es: {}'.format(num_inconsistentes))

El número de filas con datos inconsistentes es: 589


**NOTA:**  El código de la celda de arriba hace lo mismo que el snippet de código mostrado a continuación:

```python
num_inconsistentes = 0
for line in datos.values:
    if ('-' in line) or ('?' in line) or ('Xx' in line):
        # Suma uno si hay alguno de estos símbolos. 
        num_inconsistentes += 1
```

Calculamos el porcentaje de datos inválidos con respecto al tamaño del conjunto de datos. Note que en este punto consideramos inválidos todos aquellos datos que no son categóricos, aún no hemos evaluado las restricciones individuales de cada columna.

In [15]:
porcentaje_invalidos_general = num_inconsistentes*100/len(datos_train)
print('El porcentaje de datos inválidos es del: {:.2f}%'.format(porcentaje_invalidos_general))

El porcentaje de datos inválidos es del: 0.74%


Vemos que el porcentaje de datos con los caracteres <code>-</code>, <code>?</code> y <code>Xx</code> es bastante reducido y, por tanto, podemos eliminarlos del dataset sin ningún problema. 

In [16]:
def eliminar_basura(df, basura=['-', 'Xx', '?']):
    for b in basura:
        df.drop(df[df.values == b].index, inplace=True)

In [17]:
eliminar_basura(datos_train)

In [18]:
#datos.drop(datos[(datos.values == '-') | (datos.values == 'Xx') | (datos.values == '?')].index, inplace=True)

In [19]:
print('El tamaño del nuevo conjunto de datos es: {}'.format(len(datos_train)))

El tamaño del nuevo conjunto de datos es: 79411


De acuerdo con el diccionario del negocio, ahora las variables deben tener únicamente valores numéricos (enteros, para las categóricas o flotantes, para la variable BMI). Revisemos los tipos de datos del dataset: 

In [20]:
print(datos_train.dtypes)

HighBP                  object
HighChol                object
CholCheck               object
BMI                     object
Smoker                  object
Stroke                  object
HeartDiseaseorAttack    object
PhysActivity            object
Fruits                  object
Veggies                 object
HvyAlcoholConsump       object
AnyHealthcare           object
NoDocbcCost             object
GenHlth                 object
MentHlth                object
PhysHlth                object
DiffWalk                object
Sex                     object
Age                     object
Education               object
Income                  object
Diabetes_012            object
dtype: object


Podemos ver que todas las columnas son de tipo object; y esto se debe, seguramente, a que hay algunos números que está tomando como tal y, otros, como strings. Revisemos, por ejemplo, la columna <code>Diabetes_012</code>:

In [21]:
print(datos_train['Diabetes_012'].unique())

['0' '2' '1' nan]


Confirmamos entonces que el conjunto de datos está tomando algunos valores numéricos como strings. Convertimos, entonces, estas columnas a datos numéricos. Todas las convertimos a enteros, excepción hecha de la columna BMI que, por el negocio, puede ser cualquier valor real. Además, nos percatamos de que hay algunos datos que son NaN (i.e. Not a Number). De ellos nos encargaremos más adelante.

De esta forma, convertimos todas las columnas primero a <code>float</code>, porque no podemos convertir las  columnas a <code>int</code> sin lidiar con los valores nulos. En resumen, primero pasaremos los datos a <code>float</code>, luego nos ocuparemos de los valores NaN y finalmente sí convertiremos todo a <code>int</code> (a excepción de BMI).

In [22]:
# Primero convertimos todas las columnas a float
datos_train = datos_train.astype(dtype='float64')

Empecemos a lidiar con los valores nulos. Veamos primero, en total, cuantos valores nulos hay en el dataset y que porcentaje representan estos:

In [23]:
num_NaN=datos_train.isnull().any(axis = 1).sum()
porcentaje= num_NaN/datos_train.shape[0] *100
print('Hay : {}'.format(num_NaN), "datos NaN, lo cual representa el {:.2f}%".format(porcentaje), "de datos train.")     

Hay : 267 datos NaN, lo cual representa el 0.34% de datos train.


Aunque pueden haber filas con más de un valor NaN, sabemos que el máximo valor posible de filas con valores NaN es de 0.34%. Como este porcentaje es muy pequeño, procederemos a eliminar estas filas de datos train.



In [24]:
datos_train=datos_train.dropna()
print('El tamaño del nuevo conjunto de datos es: {}'.format(len(datos_train)))

El tamaño del nuevo conjunto de datos es: 79144


Como ya tratamos los datos NaN, ahora podemos convertir las columnas a <code> int </code> a excepcion de BMI

In [25]:
datos_train_BMI= datos_train['BMI']

datos_train = datos_train.drop(['BMI'], axis=1)
datos_train = datos_train.astype(dtype='int32')
datos_train = datos_train.assign(BMI = datos_train_BMI)
datos_train.dtypes



HighBP                    int32
HighChol                  int32
CholCheck                 int32
Smoker                    int32
Stroke                    int32
HeartDiseaseorAttack      int32
PhysActivity              int32
Fruits                    int32
Veggies                   int32
HvyAlcoholConsump         int32
AnyHealthcare             int32
NoDocbcCost               int32
GenHlth                   int32
MentHlth                  int32
PhysHlth                  int32
DiffWalk                  int32
Sex                       int32
Age                       int32
Education                 int32
Income                    int32
Diabetes_012              int32
BMI                     float64
dtype: object

Ahora, revisaremos cada una de las columnasa para encontrar si estas tienen errores respecto a los rangos dados en el diccionario del conjunto de datos.

Empezaremos revisando las columnas que son binarias (1 o 0 ). Para esto, revisaremos el valor máximo y mínimo de cada columna, para saber cuales de ellas tienen valores fuera del rango definido en el diccionario

In [26]:
 datos_train['HighBP'].unique()

array([0, 1])

In [27]:
lista_binarias=['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']
lista_unicas=[]
for i in lista_binarias:
    lista_unicas.append(datos_train['HighBP'].unique())

print(lista_unicas)

[array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1])]


Con este resultado, podemos concluir que las variables que tienen posibles valores binarios, no tienen valores por fuera de su rango. 

Procederemos a mirar las variables no binarias.


In [28]:
lista_binarias=['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education', 'Income']
for i in lista_binarias:
    print("<",i,"> - Max: {}".format(datos_train[i].max()), " Min: {}".format(datos_train[i].min()))


< BMI > - Max: 98.0  Min: -35.0
< GenHlth > - Max: 5  Min: 1
< MentHlth > - Max: 36  Min: -37
< PhysHlth > - Max: 42  Min: -43
< Age > - Max: 13  Min: 1
< Education > - Max: 6  Min: 1
< Income > - Max: 8  Min: 1


Con lo obtenido y comparando con el diccionario que nos dan, nos damos cuenta que las variables que están fuera de los rangos establecidos son:
* BMI (debe ser >1 y <99): Se pasa del límite inferior
* MentHlth (debe ser >1 y <30) :Se pasa del límite inferior y superior
* PhysHlth (debe ser >1 y <30) :Se pasa del límite inferior y superior

In [29]:
fuera_rango_BMI=len(datos_train[datos_train["BMI"]<1])
fuera_rango_MentHlth=len(datos_train[datos_train["MentHlth"]<0]) + len(datos_train[datos_train["MentHlth"]>30])
fuera_rango_PhysHlth=len(datos_train[datos_train["PhysHlth"]<0]) + len(datos_train[datos_train["PhysHlth"]>30])
total= fuera_rango_BMI + fuera_rango_MentHlth + fuera_rango_PhysHlth                                                              
print("Datos fuera del rango para BMI: ",str(fuera_rango_BMI))

print("Datos fuera del rango para MentHlth: ", str(fuera_rango_MentHlth))

print("Datos fuera del rango para PhysHlth: ", str(fuera_rango_PhysHlth))
                                                                                 
print("En total hay {} datos fuera del rango".format(total))
                                                                            

Datos fuera del rango para BMI:  209
Datos fuera del rango para MentHlth:  242
Datos fuera del rango para PhysHlth:  302
En total hay 753 datos fuera del rango


In [30]:
porcentaje= total/datos_train.shape[0] *100
print('Hay : {}'.format(total), "datos fuera del rango, lo cual representa el {:.2f}%".format(porcentaje), "de datos train.")      

Hay : 753 datos fuera del rango, lo cual representa el 0.95% de datos train.


Aunque pueden haber filas con más de un valor fuera de rango, que el máximo valor posible de filas con valores fuera de rango es de 0.95%. Como este porcentaje es muy pequeño, procederemos a eliminar estas filas de datos train.

**Nota:** En el diccionario de los datos se indica que el rango de valores válidos para las variables MentHlth y PhystHlth es >=2 y <=29. Sin embargo, teniendo en cuenta la naturaleza y/o el significado de estas dos variables, se tomó la decisión de utilizar el rango >=0 y <=30 ya que, no existe ninguna razón por la cual una persona no pueda indicar que tuvo 0, 1 o 30 días de mala salud (física o mental) en el mes.

In [31]:
datos_train.drop(datos_train[datos_train["BMI"]<1].index, inplace=True)

datos_train.drop(datos_train[datos_train["MentHlth"]<0].index, inplace=True)
datos_train.drop(datos_train[datos_train["MentHlth"]>30].index, inplace=True)

datos_train.drop(datos_train[datos_train["PhysHlth"]<0].index, inplace=True)
datos_train.drop(datos_train[datos_train["PhysHlth"]>30].index, inplace=True)


Después de eliminar estas columnas, hacemos la comprobación de que se eliminaron correctamente.

In [32]:
fuera_rango_BMI=len(datos_train[datos_train["BMI"]<1])
fuera_rango_MentHlth=len(datos_train[datos_train["MentHlth"]<0]) + len(datos_train[datos_train["MentHlth"]>30])
fuera_rango_PhysHlth=len(datos_train[datos_train["PhysHlth"]<0]) + len(datos_train[datos_train["PhysHlth"]>30])
total= fuera_rango_BMI + fuera_rango_MentHlth + fuera_rango_PhysHlth                                                              
print("Datos fuera del rango para BMI: ",str(fuera_rango_BMI))

print("Datos fuera del rango para MentHlth: ", str(fuera_rango_MentHlth))

print("Datos fuera del rango para PhysHlth: ", str(fuera_rango_PhysHlth))
                                                                                 
print("En total hay {} datos fuera del rango".format(total))

print('El tamaño del nuevo conjunto de datos es: {}'.format(len(datos_train)))



Datos fuera del rango para BMI:  0
Datos fuera del rango para MentHlth:  0
Datos fuera del rango para PhysHlth:  0
En total hay 0 datos fuera del rango
El tamaño del nuevo conjunto de datos es: 78393


#De igual manera, se revisará los outliers que hay en estas tres variables, para esto, nos apoyaremos en #los diagramas de caja.

In [33]:
#datos_train.boxplot(column=["BMI", "MentHlth", "MentHlth"])


Ahora, vamos a revisar si hay filas duplicadas

In [34]:
duplicados=datos_train.duplicated()
suma=0
for i in duplicados:
    if i == True:
       # print (i)
        suma +=1
print ("El número de filas duplicadas es "+str (suma))

El número de filas duplicadas es 4072


Procedemos a eliminar estas filas duplicadas.

In [35]:
datos_train=datos_train.drop_duplicates()
datos_train.shape

(74321, 22)

Para finalizar esta sección de limpieza de datos, revisaremos si debemos hacer un balanceo entre las clases correspondientes a nuestra variable de interés.

**Inserte procedimiento para hacer balanceo de datos :)**


Más adelante, hacemos un <code>Pandas Profiling</code>. Ahora, debido a que corregimos el asunto de los tipos de datos y las inconsistencias, sí podemos generarlos

In [36]:
profiling = ProfileReport(datos_train)
profiling

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [ ]:
# VALORES FUERA DE RANGO
# Pipelines